In [15]:
import numpy as np
import pandas as pd
from utils import *
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Dropout, Activation
from sklearn.model_selection import train_test_split

In [3]:
df = pd.DataFrame(columns=['postText','truthClass'])
with open('data/clickbait_data.txt') as data:
    for sentence in data:
        positive_case = {'truthClass': 1, 'postText': sentence}
        df = df.append(positive_case, ignore_index = True)
df.head()

,postText,truthClass
0,Should I Get Bings\n,1
1,Which TV Female Friend Group Do You Belong In\n,1
2,"The New ""Star Wars: The Force Awakens"" Trailer...",1
3,"This Vine Of New York On ""Celebrity Big Brothe...",1
4,A Couple Did A Stunning Photo Shoot With Their...,1


In [4]:
with open('data/non_clickbait_data.txt', encoding='utf-8') as data:
    for sentence in data:
        negative_case = {'truthClass': 0, 'postText': sentence}
        df = df.append(negative_case, ignore_index = True)
df = df.sample(frac=1)
df.head()

,postText,truthClass
17690,Ban Ki-moon calls for Zimbabwe elections to be...,0
25077,Man arrested in connection with July 21 London...,0
14238,21 Healthier Eats For People Who Hate Salad\n,1
8799,This Couple Is Proof That Not All Anonymous Co...,1
662,We Know Your Favorite Disney Movie Based On Yo...,1


In [5]:
print(df[df['truthClass'] == 0].shape)
print(df[df['truthClass'] == 1].shape)

(16001, 2)
(15999, 2)


In [6]:
df['postText'].dropna(inplace=True)
df['postText'] = df['postText'].apply(cleanText)
df.head()

,postText,truthClass
17690,ban kimoon calls for zimbabwe elections to be ...,0
25077,man arrested in connection with july 21 london...,0
14238,21 healthier eats for people who hate salad\n,1
8799,this couple is proof that not all anonymous co...,1
662,we know your favorite disney movie based on yo...,1


In [49]:
df = df.drop(df[[(len(x.split()) < 3) for x in df['postText']]].index)

postText      should i get bings\n
truthClass                       1
Name: 0, dtype: object

In [7]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.6B/glove.6B.50d.txt')

In [56]:
def sentence_to_avg(sentence, word_to_vec_map):
    words = sentence.lower().split()
    avg = np.zeros((word_to_vec_map['test'].shape))
    total = 0
    length = len(words)
    for w in words:
        if w in word_to_vec_map.keys():
            total += word_to_vec_map[w]
        else:
            length -= 1
    avg = total/length
    return avg

In [80]:
def word_embedding(postText):
    avg = sentence_to_avg(postText, word_to_vec_map)
    return avg
df['embedding'] = df['postText'].apply(word_embedding)
df.head()

,postText,truthClass,embedding
17690,ban kimoon calls for zimbabwe elections to be ...,0,"[0.17156875, -0.21309725000000002, -0.113785, ..."
25077,man arrested in connection with july 21 london...,0,"[0.3160225, 0.16974799999999998, 0.17946389999..."
14238,21 healthier eats for people who hate salad\n,1,"[0.16750500000000001, -0.03890125, -0.077963, ..."
8799,this couple is proof that not all anonymous co...,1,"[0.5863433333333333, 0.10670891666666665, 0.04..."
662,we know your favorite disney movie based on yo...,1,"[0.21679309090909094, 0.32593, -0.105227181818..."


In [81]:
train, test = train_test_split(df, test_size=0.1)
X_train, Y_train = np.array(train["embedding"].tolist()), np.array(train["truthClass"].tolist())
X_test, Y_test = np.array(test["embedding"].tolist()), np.array(test["truthClass"].tolist())
print(X_train.shape)
print(X_test.shape)

(28787, 50)
(3199, 50)


In [82]:
model = Sequential()
model.add(Dense(20, input_dim=50, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [83]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [84]:
model.fit(X_train, Y_train, epochs=20, batch_size=10)

Epoch 1/20
2879/2879 [==============================] - 1s 415us/step - loss: 0.2519 - accuracy: 0.8940
Epoch 2/20
2879/2879 [==============================] - 1s 416us/step - loss: 0.1370 - accuracy: 0.9509
Epoch 3/20
2879/2879 [==============================] - 1s 419us/step - loss: 0.1198 - accuracy: 0.9556
Epoch 4/20
2879/2879 [==============================] - 1s 421us/step - loss: 0.1245 - accuracy: 0.9544
Epoch 5/20
2879/2879 [==============================] - 1s 420us/step - loss: 0.1195 - accuracy: 0.9566
Epoch 6/20
2879/2879 [==============================] - 1s 416us/step - loss: 0.1168 - accuracy: 0.9572
Epoch 7/20
2879/2879 [==============================] - 1s 419us/step - loss: 0.1192 - accuracy: 0.9555
Epoch 8/20
2879/2879 [==============================] - 1s 422us/step - loss: 0.1131 - accuracy: 0.9582
Epoch 9/20
2879/2879 [==============================] - 1s 422us/step - loss: 0.1094 - accuracy: 0.9586
Epoch 10/20
2879/2879 [==============================] - 1s 418u

In [85]:
from sklearn.metrics import classification_report
Y_pred = model.predict(X_test)
Y_pred = [1 if i > 0.5 else 0 for [i] in Y_pred]
#print(Y_pred[0:10])
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.96      0.95      0.95      1641
           1       0.95      0.96      0.95      1558

    accuracy                           0.95      3199
   macro avg       0.95      0.95      0.95      3199
weighted avg       0.95      0.95      0.95      3199



In [90]:
# headline = "Here are 10 things you may not know" 
# s = sentence_to_avg(headline, word_to_vec_map)
# #np.array(s.tolist())
# model.predict(np.array(s.tolist()))

ValueError: in user code:

    c:\users\yuanb\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\engine\training.py:1478 predict_function  *
        return step_function(self, iterator)
    c:\users\yuanb\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\engine\training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\users\yuanb\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\users\yuanb\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\users\yuanb\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_re